# Gráficos de Cleveland: CT Filtrados Cruzados por Categoría Ideológica

Este notebook genera **gráficos de Cleveland** para visualizar el Cambio de Tiempo Filtrado (CT_Filtrado) comparando candidatos de **Izquierda** vs **Derecha**, cruzado con tipo de ítem (Progresista/Conservador), agrupado por **categoría ideológica**.

## Concepto:

### CT Filtrado Cruzado:
- **Pro_Izq**: Ítems Progresistas × Candidatos de Izquierda (suma de ítems significativos)
- **Pro_Der**: Ítems Progresistas × Candidatos de Derecha
- **Con_Izq**: Ítems Conservadores × Candidatos de Izquierda
- **Con_Der**: Ítems Conservadores × Candidatos de Derecha

### Visualización:
- **Panel A**: Ítems Progresistas (Pro_Izq vs Pro_Der)
- **Panel B**: Ítems Conservadores (Con_Izq vs Con_Der)
- **Eje Y**: 6 categorías ideológicas
- **Punto azul (●)**: Candidatos de Izquierda
- **Punto rojo (●)**: Candidatos de Derecha

## Interpretación:

- **Valores más negativos**: Mayor cambio de tiempo (más lento)
- **Hipótesis**: Mayor tiempo en contenido ideológicamente afín

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.lines import Line2D
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Librerías cargadas exitosamente')

## 1. Cargar Datos

In [ ]:
# Rutas a los archivos
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Archivo_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Archivo_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar datos
df_Generales = pd.read_excel(Archivo_Generales)
df_Ballotage = pd.read_excel(Archivo_Ballotage)

dfs = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f'✓ Datos cargados:')
print(f'  - Generales: {len(df_Generales)} registros')
print(f'  - Ballotage: {len(df_Ballotage)} registros')

# Verificar variables CT_Filtrado
vars_ct_filt = [
    'Cambio_Tiempo_Filt_Pro_Izq',
    'Cambio_Tiempo_Filt_Pro_Der',
    'Cambio_Tiempo_Filt_Con_Izq',
    'Cambio_Tiempo_Filt_Con_Der'
]

for nombre, df in dfs.items():
    faltantes = [v for v in vars_ct_filt if v not in df.columns]
    if faltantes:
        print(f'  ⚠️  {nombre}: Faltan variables {faltantes}')
    else:
        print(f'  ✅ {nombre}: Todas las variables CT_Filtrado presentes')

## 2. Preparar Datos para Gráfico

In [ ]:
# Definir categorías ideológicas
Categorias = [
    'Left_Wing',
    'Progressivism',
    'Centre',
    'Moderate_Right_A',
    'Moderate_Right_B',
    'Right_Wing_Libertarian'
]

Etiquetas_Categorias = {
    'Left_Wing': 'Left Wing',
    'Progressivism': 'Progressivism',
    'Centre': 'Centre',
    'Moderate_Right_A': 'Moderate Right A',
    'Moderate_Right_B': 'Moderate Right B',
    'Right_Wing_Libertarian': 'Right Wing Libertarian'
}

print(f'Categorías ideológicas: {len(Categorias)}')
for cat in Categorias:
    print(f'  - {Etiquetas_Categorias[cat]}')

In [ ]:
# Preparar datos por categoría para cada tipo (Progresistas y Conservadores)
def Preparar_Datos_CT_Filtrado(df, categorias, var_izq, var_der, tipo_item):
    """
    Calcula promedios de CT_Filtrado por categoría.
    Incluye test t pareado para significancia estadística.
    """
    datos = []
    
    for categoria in categorias:
        df_cat = df[df['Categoria_PASO_2023'] == categoria]
        
        if len(df_cat) > 0:
            media_izq = df_cat[var_izq].mean()
            media_der = df_cat[var_der].mean()
            diferencia = media_izq - media_der
            
            # Test t pareado para significancia
            datos_pareados = df_cat[[var_izq, var_der]].dropna()
            if len(datos_pareados) > 1:
                from scipy.stats import ttest_rel
                try:
                    t_stat, p_valor = ttest_rel(datos_pareados[var_izq], 
                                                datos_pareados[var_der])
                    
                    # Determinar asteriscos de significancia
                    if p_valor < 0.001:
                        sig = '***'
                    elif p_valor < 0.01:
                        sig = '**'
                    elif p_valor < 0.05:
                        sig = '*'
                    else:
                        sig = ''
                except:
                    p_valor = np.nan
                    sig = ''
            else:
                p_valor = np.nan
                sig = ''
            
            datos.append({
                'Categoria': categoria,
                'Etiqueta': Etiquetas_Categorias[categoria],
                'Tipo_Item': tipo_item,
                'n': len(df_cat),
                'CT_Izquierda': media_izq,
                'CT_Derecha': media_der,
                'Diferencia': diferencia,
                'p_valor': p_valor,
                'Significancia': sig
            })
    
    return pd.DataFrame(datos)

# Preparar datos para ambos datasets y ambos tipos
datos_graficos = {}

for nombre, df in dfs.items():
    # Ítems Progresistas
    df_prog = Preparar_Datos_CT_Filtrado(
        df, Categorias,
        'Cambio_Tiempo_Filt_Pro_Izq',
        'Cambio_Tiempo_Filt_Pro_Der',
        'Progresistas'
    )
    
    # Ítems Conservadores
    df_cons = Preparar_Datos_CT_Filtrado(
        df, Categorias,
        'Cambio_Tiempo_Filt_Con_Izq',
        'Cambio_Tiempo_Filt_Con_Der',
        'Conservadores'
    )
    
    datos_graficos[nombre] = {
        'Progresistas': df_prog,
        'Conservadores': df_cons
    }
    
    print(f'\n{nombre}:')
    print('  Progresistas:')
    print(df_prog[['Etiqueta', 'n', 'CT_Izquierda', 'CT_Derecha', 'Diferencia', 'Significancia']])
    print('\n  Conservadores:')
    print(df_cons[['Etiqueta', 'n', 'CT_Izquierda', 'CT_Derecha', 'Diferencia', 'Significancia']])

## 3. Función para Crear Gráfico Cleveland Comparativo

In [ ]:
def Crear_Grafico_Cleveland_CT_Filtrado_Comparativo(df_prog, df_cons, titulo, nombre_archivo=None, carpeta_destino='Graficos_Cleveland'):
    """
    Crea gráfico con 2 paneles: Progresistas vs Conservadores.
    Incluye asteriscos para indicar significancia estadística.
    """
    
    # Crear carpeta si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    # Crear figura con 2 paneles
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Determinar límites comunes del eje X
    min_val = min(
        df_prog['CT_Izquierda'].min(), df_prog['CT_Derecha'].min(),
        df_cons['CT_Izquierda'].min(), df_cons['CT_Derecha'].min()
    )
    max_val = max(
        df_prog['CT_Izquierda'].max(), df_prog['CT_Derecha'].max(),
        df_cons['CT_Izquierda'].max(), df_cons['CT_Derecha'].max()
    )
    margen = (max_val - min_val) * 0.1
    
    # Función helper para dibujar un panel
    def Dibujar_Panel(ax, df, titulo_panel, mostrar_ylabel):
        # Ordenar por diferencia
        df_sorted = df.sort_values('Diferencia', ascending=True).reset_index(drop=True)
        y_positions = np.arange(len(df_sorted))
        
        # Dibujar líneas
        for idx, row in df_sorted.iterrows():
            izq_val = row['CT_Izquierda']
            der_val = row['CT_Derecha']
            diferencia = row['Diferencia']
            
            # Color según diferencia
            if diferencia > 0.5:
                color_linea = '#3498db'  # Azul
                alpha = 0.7
            elif diferencia < -0.5:
                color_linea = '#e74c3c'  # Rojo
                alpha = 0.7
            else:
                color_linea = '#95a5a6'  # Gris
                alpha = 0.4
            
            ax.plot([der_val, izq_val], [idx, idx],
                    color=color_linea, linewidth=2, alpha=alpha, zorder=1)
        
        # Dibujar puntos
        ax.scatter(df_sorted['CT_Izquierda'], y_positions,
                   s=150, c='#3498db', marker='o',
                   edgecolors='white', linewidths=2,
                   label='Izquierda', zorder=3, alpha=0.9)
        
        ax.scatter(df_sorted['CT_Derecha'], y_positions,
                   s=150, c='#e74c3c', marker='o',
                   edgecolors='white', linewidths=2,
                   label='Derecha', zorder=3, alpha=0.9)
        
        # Línea vertical en x=0
        ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3, zorder=0)
        
        # Configurar ejes con asteriscos de significancia
        ax.set_yticks(y_positions)
        etiquetas_con_sig = [f"{row['Etiqueta']} {row['Significancia']}" if row['Significancia'] else row['Etiqueta'] 
                             for _, row in df_sorted.iterrows()]
        ax.set_yticklabels(etiquetas_con_sig, fontsize=11)
        
        ax.set_xlabel('CT Filtrado (segundos)', fontsize=12, fontweight='bold')
        if mostrar_ylabel:
            ax.set_ylabel('Categoría Ideológica', fontsize=12, fontweight='bold')
        
        ax.set_title(titulo_panel, fontsize=13, fontweight='bold', pad=15)
        
        # Grid
        ax.grid(True, axis='x', alpha=0.3, linestyle=':', linewidth=0.5)
        ax.set_axisbelow(True)
        
        # Ajustar límites X
        ax.set_xlim(min_val - margen, max_val + margen)
    
    # Panel A: Progresistas
    Dibujar_Panel(ax1, df_prog, 'A) Ítems Progresistas', True)
    
    # Panel B: Conservadores
    Dibujar_Panel(ax2, df_cons, 'B) Ítems Conservadores', False)
    
    # Leyenda común
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db',
               markersize=11, label='Candidatos Izquierda', markeredgecolor='white', markeredgewidth=1.5),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#e74c3c',
               markersize=11, label='Candidatos Derecha', markeredgecolor='white', markeredgewidth=1.5),
        Line2D([0], [0], color='#3498db', linewidth=2.5, label='Más Izq (>0.5s)'),
        Line2D([0], [0], color='#95a5a6', linewidth=2.5, label='Similar (±0.5s)'),
        Line2D([0], [0], color='#e74c3c', linewidth=2.5, label='Más Der (<-0.5s)'),
        Line2D([0], [0], marker='', color='w', label='* p<0.05, ** p<0.01, *** p<0.001')
    ]
    
    fig.legend(handles=legend_elements, loc='upper center', ncol=6,
               fontsize=11, framealpha=0.95, edgecolor='gray',
               bbox_to_anchor=(0.5, 0.98))
    
    # Título general
    fig.suptitle(titulo, fontsize=14, fontweight='bold', y=0.995)
    
    # Ajustar layout
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # Guardar
    if nombre_archivo:
        ruta_completa = os.path.join(carpeta_destino, f'{nombre_archivo}.png')
        plt.savefig(ruta_completa, dpi=300, bbox_inches='tight', facecolor='white')
        print(f'✅ Gráfico guardado: {ruta_completa}')
    
    plt.show()
    
    return fig, (ax1, ax2)

## 4. Gráficos por Dataset

In [ ]:
# Gráfico para Generales
print('Generando gráfico para Generales...\n')
fig_gen, (ax1_gen, ax2_gen) = Crear_Grafico_Cleveland_CT_Filtrado_Comparativo(
    datos_graficos['Generales']['Progresistas'],
    datos_graficos['Generales']['Conservadores'],
    titulo='CT Filtrado Cruzado: Tipo de Ítem × Ideología de Candidato - Generales',
    nombre_archivo='Cleveland_CT_Filtrado_Cruzado_Generales'
)

In [ ]:
# Gráfico para Ballotage
print('Generando gráfico para Ballotage...\n')
fig_bal, (ax1_bal, ax2_bal) = Crear_Grafico_Cleveland_CT_Filtrado_Comparativo(
    datos_graficos['Ballotage']['Progresistas'],
    datos_graficos['Ballotage']['Conservadores'],
    titulo='CT Filtrado Cruzado: Tipo de Ítem × Ideología de Candidato - Ballotage',
    nombre_archivo='Cleveland_CT_Filtrado_Cruzado_Ballotage'
)

## 4.1. Análisis de Significancia Estadística

In [ ]:
print('='*70)
print('ANÁLISIS DE SIGNIFICANCIA ESTADÍSTICA')
print('Test t Pareado: CT Izquierda vs Derecha (por Tipo de Ítem)')
print('='*70)

for nombre, datos in datos_graficos.items():
    print(f'\n📊 {nombre}:')
    print('='*70)
    
    for tipo, df_datos in datos.items():
        print(f'\n  {tipo}:')
        print('  ' + '-'*66)
        
        # Filtrar categorías con significancia
        sig_cats = df_datos[df_datos['Significancia'] != ''].copy()
        
        if len(sig_cats) > 0:
            print(f'\n  ✅ Categorías con diferencias significativas: {len(sig_cats)}/{len(df_datos)}')
            
            # Separar por dirección
            sig_mas_izq = sig_cats[sig_cats['Diferencia'] > 0]
            sig_mas_der = sig_cats[sig_cats['Diferencia'] < 0]
            
            if len(sig_mas_izq) > 0:
                print(f'\n    Más tiempo en IZQUIERDA ({len(sig_mas_izq)} categorías):')
                for _, row in sig_mas_izq.iterrows():
                    print(f"      {row['Significancia']:3s} {row['Etiqueta']:25s} | Dif: {row['Diferencia']:6.3f}s | p={row['p_valor']:.4f}")
            
            if len(sig_mas_der) > 0:
                print(f'\n    Más tiempo en DERECHA ({len(sig_mas_der)} categorías):')
                for _, row in sig_mas_der.iterrows():
                    print(f"      {row['Significancia']:3s} {row['Etiqueta']:25s} | Dif: {row['Diferencia']:6.3f}s | p={row['p_valor']:.4f}")
        else:
            print('\n  ❌ No hay categorías con diferencias significativas (p < 0.05)')
        
        # Categorías no significativas
        no_sig = df_datos[df_datos['Significancia'] == '']
        if len(no_sig) > 0:
            print(f'\n  ⚪ Categorías sin diferencias significativas: {len(no_sig)}/{len(df_datos)}')

print('\n' + '='*70)
print('Niveles de significancia: * p<0.05, ** p<0.01, *** p<0.001')
print('='*70)

## 5. Análisis Estadístico

In [ ]:
print('='*70)
print('ANÁLISIS ESTADÍSTICO: CT FILTRADO CRUZADO')
print('='*70)

for nombre, datos in datos_graficos.items():
    print(f'\n📊 {nombre}:')
    print('='*70)
    
    for tipo, df_datos in datos.items():
        print(f'\n  {tipo}:')
        print('  ' + '-'*66)
        
        print(f'\n  Estadísticas Generales:')
        print(f'    Promedio CT_Izquierda: {df_datos["CT_Izquierda"].mean():.3f} seg')
        print(f'    Promedio CT_Derecha: {df_datos["CT_Derecha"].mean():.3f} seg')
        print(f'    Diferencia promedio: {df_datos["Diferencia"].mean():.3f} seg')
        
        # Test t pareado
        t_stat, p_value = stats.ttest_rel(df_datos['CT_Izquierda'], df_datos['CT_Derecha'])
        
        print(f'\n  Test t Pareado:')
        print(f'    Estadístico t: {t_stat:.4f}')
        print(f'    Valor p: {p_value:.4f}')
        
        if p_value < 0.05:
            print(f'    ✅ SIGNIFICATIVO (p < 0.05)')
            if df_datos['Diferencia'].mean() > 0:
                print(f'       Mayor tiempo en candidatos de Izquierda')
            else:
                print(f'       Mayor tiempo en candidatos de Derecha')
        else:
            print(f'    ❌ NO SIGNIFICATIVO (p ≥ 0.05)')

print('\n' + '='*70)

## 6. Interpretación Teórica

In [ ]:
print('='*70)
print('INTERPRETACIÓN TEÓRICA')
print('='*70)

for nombre, datos in datos_graficos.items():
    print(f'\n📊 {nombre}:')
    print('='*70)
    
    dif_prog = datos['Progresistas']['Diferencia'].mean()
    dif_cons = datos['Conservadores']['Diferencia'].mean()
    
    print(f'\n  Ítems Progresistas:')
    print(f'    Diferencia promedio (Izq - Der): {dif_prog:.3f} seg')
    if dif_prog > 0.5:
        print(f'    ✅ ESPERADO: Candidatos de Izquierda tardan MÁS en ítems progresistas')
        print(f'       (Mayor procesamiento de contenido ideológicamente afín)')
    elif dif_prog < -0.5:
        print(f'    ⚠️  INESPERADO: Candidatos de Derecha tardan MÁS en ítems progresistas')
    else:
        print(f'    ➖ Sin diferencia clara entre candidatos')
    
    print(f'\n  Ítems Conservadores:')
    print(f'    Diferencia promedio (Izq - Der): {dif_cons:.3f} seg')
    if dif_cons < -0.5:
        print(f'    ✅ ESPERADO: Candidatos de Derecha tardan MÁS en ítems conservadores')
        print(f'       (Mayor procesamiento de contenido ideológicamente afín)')
    elif dif_cons > 0.5:
        print(f'    ⚠️  INESPERADO: Candidatos de Izquierda tardan MÁS en ítems conservadores')
    else:
        print(f'    ➖ Sin diferencia clara entre candidatos')
    
    print(f'\n  Patrón de Interacción (Tipo × Ideología):')
    if (dif_prog > 0 and dif_cons < 0):
        print(f'    ✅ INTERACCIÓN ESPERADA:')
        print(f'       - Izquierda tarda más en Progresistas')
        print(f'       - Derecha tarda más en Conservadores')
        print(f'       → Procesamiento especializado según afinidad ideológica')
    elif (dif_prog < 0 and dif_cons > 0):
        print(f'    ⚠️  INTERACCIÓN INVERSA:')
        print(f'       - Derecha tarda más en Progresistas')
        print(f'       - Izquierda tarda más en Conservadores')
        print(f'       → Patrón opuesto al esperado')
    elif (dif_prog > 0 and dif_cons > 0):
        print(f'    ➖ SIN INTERACCIÓN (efecto principal de Izquierda):')
        print(f'       - Izquierda tarda más en ambos tipos')
    elif (dif_prog < 0 and dif_cons < 0):
        print(f'    ➖ SIN INTERACCIÓN (efecto principal de Derecha):')
        print(f'       - Derecha tarda más en ambos tipos')
    else:
        print(f'    ➖ Sin patrón claro')

print('\n' + '='*70)

## 7. Guardar Tablas Resumen

In [ ]:
# Crear carpeta de salida
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_Cleveland')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar tablas
for nombre, datos in datos_graficos.items():
    for tipo, df_datos in datos.items():
        archivo = os.path.join(Carpeta_Salida, f'Resumen_CT_Filtrado_{tipo}_{nombre}.xlsx')
        df_datos.to_excel(archivo, index=False)
        print(f'✅ Guardado: {archivo}')

## 8. Resumen Final

In [ ]:
print('='*70)
print('RESUMEN: CT FILTRADO CRUZADO')
print('='*70)

print('\n📊 Análisis completado:')
print(f'  - Datasets analizados: 2 (Generales y Ballotage)')
print(f'  - Tipos de ítems: 2 (Progresistas y Conservadores)')
print(f'  - Categorías por dataset: {len(Categorias)}')
print(f'  - Gráficos generados: 2 (comparativos con 2 paneles cada uno)')

print('\n📁 Archivos generados:')
print('  Gráficos:')
print('    - Cleveland_CT_Filtrado_Cruzado_Generales.png')
print('    - Cleveland_CT_Filtrado_Cruzado_Ballotage.png')
print('  Tablas Excel:')
print('    - Resumen_CT_Filtrado_Progresistas_Generales.xlsx')
print('    - Resumen_CT_Filtrado_Conservadores_Generales.xlsx')
print('    - Resumen_CT_Filtrado_Progresistas_Ballotage.xlsx')
print('    - Resumen_CT_Filtrado_Conservadores_Ballotage.xlsx')

print('\n💡 Interpretación:')
print('  - CT Filtrado usa solo ítems que fueron significativos en Kruskal-Wallis')
print('  - Hipótesis: Mayor tiempo en contenido ideológicamente afín')
print('  - Patrón esperado: Interacción Tipo_Ítem × Ideología_Candidato')

print('\n' + '='*70)
print('✅ ANÁLISIS COMPLETADO')
print('='*70)